 ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

In [27]:
# The First I will Import library python that O need 
import pandas as pd
import cassandra
import re #Regular expression 
import os # operating system
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
print(os.getcwd())

/home/workspace


In [29]:
filepath = os.getcwd() + '/event_data'
print(filepath)

/home/workspace/event_data


In [30]:
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [47]:
full_data_rows_list = []

for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new1.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [48]:
print(len(full_data_rows_list))

685


In [49]:
# check the number of rows in csv file
with open('event_datafile_new1.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

597


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new1.csv</font>, located within the Workspace directory.  The event_datafile_new1.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [50]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [51]:
session.execute("""CREATE keyspace IF NOT EXISTS sparkify
                    WITH replication = { 'class':'SimpleStrategy', 'replication_factor':1}""")

#### Set Keyspace

In [52]:
session.set_keyspace('sparkify')

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [53]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
session.execute("""
        CREATE TABLE IF NOT EXISTS session_songs 
        (sessionId int ,itemInSession int , artist text,song_title text ,song_length float
        ,PRIMARY KEY (sessionId,itemInSession))""")

In [54]:
file = 'event_datafile_new1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId ,itemInSession , artist,song_title ,song_length) "
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute (query,(int(sessionId),int(itemInSession),artist_name,song,float(length)))  
            

#### Do a SELECT to verify that the data have been inserted into each table

In [55]:
rows=session.execute("""SELECT artist,song_title,song_length FROM session_songs where sessionId=338 and itemInSession =4""")

for i in rows:
    print(i.artist,i.song_title,i.song_length)

### 2. Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [56]:
session.execute("""      

        create table if not exists event_one
            
            (artist text, 
            song text, 
            first_name text, 
            last_name text,
            user_id int,
            session_id int,
            item_in_session int, 
            primary key ((user_id, session_id), item_in_session))
""")

In [57]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_one (artist ,song , first_name,last_name ,user_id,session_id,item_in_session) "
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
            

In [58]:

rows = session.execute("""
                            select artist , song , first_name , last_name
                            from event_one
                            where user_id = 10 and session_id = 182
                            order by item_in_session 

""")
for row in rows:
    print (row.artist , row.song , row.first_name , row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz




### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [43]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""      

        create table if not exists Songes
            
            (song text, 
             user_id int,
            first_name text, 
            last_name text,
            primary key (song, user_id))
""")        

In [44]:
file = 'event_datafile_new1.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO Songes (song ,user_id, first_name,last_name) "
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]), line[1], line[4]))
            

In [45]:

rows = session.execute("""
                            select first_name , last_name
                            from Songes
                             WHERE song = 'All Hands Against His Own'
               
""")
for row in rows:
    print (row.first_name , row.last_name)

Sara Johnson


### Drop the tables before closing out the sessions

In [46]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE session_songs")
session.execute("DROP TABLE event_one")
session.execute("DROP TABLE Songes")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()